## 教師データ収集用スクリプト

衝突回避を学習するためのデータを収集します。
jetbotは走りながらカメラから画像を取り込み、画像をもとに次の行動を決定します。
このタスクでjetbotに必要なことは、障害物がなければまっすぐ進み、障害物があれば左折して回避することの二つです。
よって、「障害物がない写真」、「障害物がある写真」の2パターンを用意する必要があります。
このスクリプトでは、実際にjetbotを遠隔操作で走らせ、直進する(=障害物がない)か左折する(=障害物がある)かを選択します。
直進中に撮られた写真には直進のラベルがつけられ、左折中に撮られた写真には左折のラベルがつけられます。
AIはこのデータを使って、どれが直進していい画像なのかを学習することになります。

AIに使うニューラルネットワークの入力サイズに合わせて、カメラの画像を224px×224pxで保存します。

In [8]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera
cap_image = widgets.Image(format='jpeg', width=224, height=224)

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)


画像を保存するフォルダを設定します。
障害物のない画像は`dataset/free`に、障害物のある画像は`dataset/blocked`に保存します。

In [2]:
import os

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


操作ボタンを定義します。

In [10]:
button_layout = widgets.Layout(width='200px', height='48px')
counter_layout = widgets.Layout(width='200px', height='30px')
text_layout = widgets.Layout(width='256px', height='30px')

free_count = widgets.IntText(layout=counter_layout, value=len(os.listdir(free_dir)),description='free')
blocked_count = widgets.IntText(layout=counter_layout, value=len(os.listdir(blocked_dir)),description='blocked')

capture_free_button = widgets.Button(description='Capture free', button_style='success', layout=button_layout)
capture_blocked_button = widgets.Button(description='Capture blocked', button_style='danger', layout=button_layout)

free_button = widgets.Button(description='Forward', button_style='success', layout=button_layout)
stop_button = widgets.Button(description='Stop', button_style='info', layout=button_layout)
blocked_button = widgets.Button(description='Left', button_style='danger', layout=button_layout)
 
speed_slider=widgets.FloatSlider(description='speed',value=0.3,min=0,max=1,step=0.01)
turn_slider=widgets.FloatSlider(description='turn left',value=0.3,min=0,max=1,step=0.01)

zip_text=widgets.Text(layout=text_layout, value="dataset",description='zipfile:')

各フォルダに写真を保存するメソッドを定義しています。

In [12]:
from uuid import uuid1

def save_snapshot(directory):
    cap_image=image
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(cap_image.value)
def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))

capture_free_button.on_click(lambda x: save_free())
capture_blocked_button.on_click(lambda x: save_blocked())

モータを制御するため、Robotインスタンスを生成します。

In [6]:
from jetbot import Robot
robot = Robot()

ModuleNotFoundError: No module named 'jetbot'

ボタンを押したときのjetbotの挙動を定義します。
前進の時は前進し続けながら一定間隔で画像を保存します。
左折の時は、写真を撮り、少し左へ方向転換して止まります。
これは、直進の時はあまり操作が必要ないので効率性を重視し、障害物を回避するときには慎重に操作する必要があるので確実性を重視したためです。

In [14]:
import time

isFree = False
isRunning = False
stepcount = 0

def set_forward(change):
    global stepcount, isFree, isRunning, robot
    stepcount = 0
    isRunning = True
    isFree = True

def set_left(change):
    global stepcount, isFree, isRunning, robot
    robot.stop()
    save_blocked()
    robot.left(turn_slider.value)
    stepcount = 0
    isRunning = True
    isFree = False
            

def update(change):
    global stepcount, isFree, isRunning, robot
    time.sleep(0.001)
    if(isRunning):
        if(isFree):
            if(stepcount % 25 == 0):
                robot.stop()
                save_free()
                robot.forward(speed_slider.value)
        else:
            if(stepcount % 6 == 5):
                isRunning = False
                robot.stop()
        stepcount += 1
        
    
def stop(change):
    global isRunning, robot
    isRunning = False
    robot.stop()

free_button.on_click(set_forward)
blocked_button.on_click(set_left)
stop_button.on_click(stop)

UIを表示させます。UIを構成するのはカメラからのイメージと操作パネルです。
操作パネル上段には、直進ボタンと直進フォルダに保存された画像の数が表示されます。
また、中段には左折ボタンと左折フォルダに保存された画像の数を表示されます。
最後に止まるボタンを表示します。

In [13]:
display(cap_image)
Counter=widgets.HBox([free_count,blocked_count])
Capture=widgets.HBox([capture_free_button,capture_blocked_button])
Foward_Left=widgets.HBox([free_button,blocked_button])
display(Counter)
display(Capture)
display(Foward_Left)
display(stop_button)
display(speed_slider)
display(turn_slider)
#display(zip_text)

Image(value=b'', format='jpeg', height='224', width='224')

Button(button_style='info', description='Stop', layout=Layout(height='48px', width='128px'), style=ButtonStyle…

FloatSlider(value=0.3, description='speed', max=1.0, step=0.01)

FloatSlider(value=0.3, description='turn left', max=1.0, step=0.01)

NameError: name 'robot' is not defined

NameError: name 'robot' is not defined

カメラに先ほど定義したメソッドを紐づけます。
カメラの画像が更新されるたびにupdateメソッドが呼ばれることになります。
これを実行することで、ボタンを押すとjetbotが走り出すようになります。

In [15]:
camera.observe(update, names='value')

NameError: name 'camera' is not defined

以下はデータ収集の終了後に実行する処理です。
updateメソッドとカメラとの関連付けを外します。
また、jetbotを停止させます。

In [10]:
camera.unobserve(update, names='value')
robot.stop()

NameError: name 'camera' is not defined

datasetフォルダの中身をzipファイルにして保存し、ダウンロード可能にします。
jetbot上で学習も行う場合は不要です。

In [9]:
!zip -r -q dataset.zip dataset

SyntaxError: invalid syntax (<ipython-input-9-0015766eaea9>, line 1)